# หา cosine similarity ของข่าวบน cofact (ทดสอบกับข่าวจำนวนมาก ครั้งที่ 1) ระหว่างข้อความดั้งเดิมกับหัวข่าวอ้างอิง แบบ Bags of word + TF-IDF

In [2]:
import pythainlp
import numpy as np
import math
import nltk
import re
import pandas as pd 
import seaborn as sns
import random 
import json
import os
import matplotlib.pyplot as plt

from pythainlp import word_tokenize
from pythainlp import corpus
from pythainlp.corpus import thai_stopwords
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
from string import punctuation

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from scipy import spatial

nltk.download('words')
th_stop = thai_stopwords()
p_stemmer = PorterStemmer()

def split_word(text):
    tokens = word_tokenize(text,engine='icu')
    
    # Remove stop words ภาษาไทย และอักขระพิเศษ
    tokens = [i for i in tokens if not i in th_stop and not i in punctuation and not i in ['“','”','‘','’']] 
    
    # Thai transform to root word
    # tokens_temp=[]
    # for i in tokens:
    #     w_syn = wordnet.synsets(i)
    #     if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
    #         tokens_temp.append(w_syn[0].lemma_names('tha')[0])
    #     else:
    #         tokens_temp.append(i)
    
    # tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

def find_cosine_similarity(original_text_vector, headline_news_vector, first_text_index, second_text_index):
    sum = 0 #upper text of cosine similarity
    sum_a = 0
    sum_b = 0
    total_words_number = original_text_and_headline_news_vector.shape[1]

    for index in range(total_words_number):
        if (original_text_vector[first_text_index, index] == 0) and (headline_news_vector[second_text_index, index] ==0):
            continue
        sum_a += original_text_vector[first_text_index, index]**2
        sum_b += headline_news_vector[second_text_index, index]**2

        sum += original_text_vector[first_text_index, index]*headline_news_vector[second_text_index, index]

    # print('This is sum (upperpart) :',sum)
    
    squareroot_sum_a = math.sqrt(sum_a)
    squareroot_sum_b = math.sqrt(sum_b)
    lowerpart = squareroot_sum_a * squareroot_sum_b

    # print('This is lowerpart :',lowerpart)
    return (sum/lowerpart)

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
def read_cofact_detail():
    # Opening JSON file
    root_path = os.getcwd()
    path = os.path.join(root_path, '..\Scrapy\Scrapy_project\spiders\\fetch file\cofact_detail.json')
    f = open(path, encoding="utf8")
    
    # returns JSON object as
    # a dictionary
    cofact_original_text_data = json.load(f)

    # Closing file
    f.close()
    return cofact_original_text_data

In [4]:
print(os.getcwd())

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process


In [5]:
cofact_original_text_data = read_cofact_detail()

In [6]:
def get_cofact_original_text_info(original_text_data): #สำหรับดึงข้อความดั้งเดิม (header) และลิ้งของข้อความดั้งเดิม (reference) จากเว็บไซต์ cofact
    original_text_list = []

    for original_text in range(len(original_text_data)):
        tmp = []
        tmp.append(str(original_text_data[original_text]['header']) + str(original_text_data[original_text]['content']))
        tmp.append(original_text_data[original_text]['link'])
        original_text_list.append(tmp)

    return original_text_list

In [7]:
type(cofact_original_text_data[0]['header'])

NoneType

In [8]:
type(cofact_original_text_data[0]['content'])

NoneType

In [9]:
cofact_original_text_list = get_cofact_original_text_info(cofact_original_text_data)

In [10]:
cofact_original_text_list

[['NoneNone', 'https://cofact.org/article/3f8tsdv1edj3z'],
 ['Noneวัคซีนโมเดอร์นาเข้าไทยจำนวน 560,200 โดส สภากาชาดไทยได้วัคซีนจํานวน 112,040 โดส จริงหรือ',
  'https://cofact.org/article/1yojsfkrd6uqe'],
 ['None4 ใน 10 คนที่ฉีดวัคซีน Pfizer จะหัวล้าน-เซ็กซ์เสื่อม จริงหรือ?',
  'https://cofact.org/article/352jxxusnfy9r'],
 ['NoneNone', 'https://cofact.org/article/1dfyape95t604'],
 ['อย. ยกเลิก เลขสารบบอาหาร ผลิตภัณฑ์เสริมอาหาร คอลลาเจน พลัส กลิ่นมิกซ์ เบอร์รี่มะละกอ ได้รับการคัดเลือกจาก WHO (องค์การอนามัยโลก) เป็นปีที่ 2 ติดต่อกันให้เป็นผลไม้ที่มีคุณค่าทางโภชนาการสูงสุด',
  'https://cofact.org/article/24y8wj4x5a4vv'],
 ['Noneจริงไหมตอนนี้ชาวนาขายข้าวต่อกิโลได้เท่ากับค่าบะหมี่กึ่งสำเร็จรูปซองเดียว 😢',
  'https://cofact.org/article/2h4lxtqjm8dr6'],
 ['NoneNone', 'https://cofact.org/article/1vgxwka8atjg'],
 ['NoneNone', 'https://cofact.org/article/2zcpe21q7nins'],
 ['ธนาคารออมสินช่วยผู้ประสบภัยน้ำท่วมภาคใต้ ให้กู้ฉุกเฉินไม่คิดดอกเบี้ย 1 ปี และพักชำระหนี้ 3 เดือน จริงหรือ?วัคซีนโควิด -9 จะทำ

In [11]:
cofact_original_text_list[0][1]

'https://cofact.org/article/3f8tsdv1edj3z'

In [12]:
def read_cofact_refer():
    # Opening JSON file
    root_path = os.getcwd()
    path = os.path.join(root_path, '..\Scrapy\Scrapy_project\spiders\\fetch file\cofact_refer.json')
    f = open(path, encoding="utf8")
    
    # returns JSON object as
    # a dictionary
    cofact_refer_text_data = json.load(f)

    # Closing file
    f.close()
    return cofact_refer_text_data

In [13]:
cofact_refer_text_data = read_cofact_refer()

In [14]:
len(cofact_refer_text_data)

2206

In [15]:
def get_cofact_refer_info(cofact_refer_text_data): #สำหรับดึงข้อความดั้งเดิม (header) และลิ้งของข้อความดั้งเดิม (reference) จากเว็บไซต์ cofact
    refer_text_list = []

    for refer_text in range(len(cofact_refer_text_data)):
        tmp = []
        tmp.append(cofact_refer_text_data[refer_text]['header'])
        tmp.append(cofact_refer_text_data[refer_text]['reference'])
        refer_text_list.append(tmp)

    return refer_text_list

In [16]:
cofact_refer_text_list = get_cofact_refer_info(cofact_refer_text_data)

In [17]:
cofact_refer_text_list

[['เบื้องลึก ราคาข้าวตกต่ำ ชาวนาไทย ตกเป็นเครื่องมือการเมือง ไร้อำนาจต่อรอง',
  'https://cofact.org/article/2h4lxtqjm8dr6'],
 ['1 พ.ย.วันแรกเปิดประเทศ นักท่องเที่ยวแห่เข้าไทยกว่า 60 เที่ยวบิน ',
  'https://cofact.org/article/iysxqcesrsi0'],
 ['ลงทะเบียนคนละครึ่ง 1.1 แสนสิทธิ์เต็มแล้ว มีสิทธิ์อยู่ รับเพิ่ม 1,500 บาท',
  'https://cofact.org/article/3upxzqi4sbvf7'],
 ['สธ.แนะฉีดวัคซีน mRNA กระตุ้นเข็ม 3 เต็มโดส ฉีดเชื้อตายไม่เหมาะกระตุ้นเชื้อตายซ้ำ',
  'https://cofact.org/article/32swjuf5cjr3g'],
 ['รถบรรทุกหยุดวิ่ง 20% ประท้วงน้ำมันแพง  ',
  'https://cofact.org/article/iysxqcesrsi0'],
 ['ชาวเน็ตแห่แชร์ วิธีการป้องกันการ “ถูกดูดเงินจากบัญชี” มีอะไรบ้างเช็กเลยที่นี่',
  'https://cofact.org/article/fwny3pfdh01d'],
 ['ข่าวปลอม! นามบัตรเคลือบยา สามารถทำให้สลบได้ในทันทีหากสัมผัส',
  'https://cofact.org/article/1l28zrz16wny'],
 ['ยอดไอเดีย! นวัตกรรม "เก้าอี้ติดขา" อยากนั่งที่ไหนก็นั่ง [ชมคลิป]',
  'https://cofact.org/article/2mifjg1fxp9ga'],
 ['ยอดโควิดสะสมสิงคโปร์ทะลุแสน ผู้ป่วยเพิ่มขึ้นสองเท่

In [18]:
len(cofact_refer_text_list)

2206

In [19]:
len(cofact_original_text_list)

4286

In [20]:
def match_cofact_news(cofact_refer_text_list, cofact_original_text_list):
    original_text_and_headline_news = []

    for refer_text in range(len(cofact_refer_text_list)):
        tmp = []
        for original_text in range(len(cofact_original_text_list)):
            if cofact_refer_text_list[refer_text][1] == cofact_original_text_list[original_text][1] and (cofact_original_text_list[original_text][0] != "NoneNone"):
                tmp.append(cofact_original_text_list[original_text][0])
                tmp.append(cofact_refer_text_list[refer_text][0])
                original_text_and_headline_news.append(tmp)
                break

    return original_text_and_headline_news
        

In [21]:
matched_cofact_news = match_cofact_news(cofact_refer_text_list, cofact_original_text_list)

In [22]:
matched_cofact_news

[['Noneจริงไหมตอนนี้ชาวนาขายข้าวต่อกิโลได้เท่ากับค่าบะหมี่กึ่งสำเร็จรูปซองเดียว 😢',
  'เบื้องลึก ราคาข้าวตกต่ำ ชาวนาไทย ตกเป็นเครื่องมือการเมือง ไร้อำนาจต่อรอง'],
 ['การฉีดวัคซีนกระตุ้นด้วยวัคซีน mRNA ครึ่งโดส หรือ 50 ไมโครกรัม  ไม่มีข้อมูลชัดเจนว่า ภูมิคุ้มกันจะอยู่ได้นานเท่าใด จึงแนะนำให้ฉีดวัคซีนเต็มโดส คือ 100 ไมโครกรัม จริงหรือไม่การฉีดวัคซีนกระตุ้นด้วยวัคซีน mRNA ครึ่งโดส หรือ 50 ไมโครกรัม  ไม่มีข้อมูลชัดเจนว่า ภูมิคุ้มกันจะอยู่ได้นานเท่าใด จึงแนะนำให้ฉีดวัคซีนเต็มโดส คือ 100 ไมโครกรัม',
  'สธ.แนะฉีดวัคซีน mRNA กระตุ้นเข็ม 3 เต็มโดส ฉีดเชื้อตายไม่เหมาะกระตุ้นเชื้อตายซ้ำ'],
 ['วุ่นหนัก! เจอแอพฯดูดเงิน ไหลออกจากบัญชี-บัตรเครดิต โดนแบบนี้นับหมื่นรายวุ่นหนัก! เจอแอพฯดูดเงิน ไหลออกจากบัญชี-บัตรเครดิต โดนแบบนี้นับหมื่นราย',
  'ชาวเน็ตแห่แชร์ วิธีการป้องกันการ “ถูกดูดเงินจากบัญชี” มีอะไรบ้างเช็กเลยที่นี่'],
 ['จริงไหม  เพื่อนฝากให้แชร์ฝากให้แชร์',
  'ข่าวปลอม! นามบัตรเคลือบยา สามารถทำให้สลบได้ในทันทีหากสัมผัส'],
 ['อย. ยกเลิกเลขสารบบอาหาร ผลิตภัณฑ์เสริมอาหาร เซเว่น เดย์ เซเว่น ดีบริษัทจ

In [23]:
original_text_and_headline_news = []
for news in matched_cofact_news:
    original_text_and_headline_news.append(news[0])
    original_text_and_headline_news.append(news[1])

In [24]:
print(original_text_and_headline_news)

['Noneจริงไหมตอนนี้ชาวนาขายข้าวต่อกิโลได้เท่ากับค่าบะหมี่กึ่งสำเร็จรูปซองเดียว 😢', 'เบื้องลึก ราคาข้าวตกต่ำ ชาวนาไทย ตกเป็นเครื่องมือการเมือง ไร้อำนาจต่อรอง', 'การฉีดวัคซีนกระตุ้นด้วยวัคซีน mRNA ครึ่งโดส หรือ 50 ไมโครกรัม  ไม่มีข้อมูลชัดเจนว่า ภูมิคุ้มกันจะอยู่ได้นานเท่าใด จึงแนะนำให้ฉีดวัคซีนเต็มโดส คือ 100 ไมโครกรัม จริงหรือไม่การฉีดวัคซีนกระตุ้นด้วยวัคซีน mRNA ครึ่งโดส หรือ 50 ไมโครกรัม  ไม่มีข้อมูลชัดเจนว่า ภูมิคุ้มกันจะอยู่ได้นานเท่าใด จึงแนะนำให้ฉีดวัคซีนเต็มโดส คือ 100 ไมโครกรัม', 'สธ.แนะฉีดวัคซีน mRNA กระตุ้นเข็ม 3 เต็มโดส ฉีดเชื้อตายไม่เหมาะกระตุ้นเชื้อตายซ้ำ', 'วุ่นหนัก! เจอแอพฯดูดเงิน ไหลออกจากบัญชี-บัตรเครดิต โดนแบบนี้นับหมื่นรายวุ่นหนัก! เจอแอพฯดูดเงิน ไหลออกจากบัญชี-บัตรเครดิต โดนแบบนี้นับหมื่นราย', 'ชาวเน็ตแห่แชร์ วิธีการป้องกันการ “ถูกดูดเงินจากบัญชี” มีอะไรบ้างเช็กเลยที่นี่', 'จริงไหม  เพื่อนฝากให้แชร์ฝากให้แชร์', 'ข่าวปลอม! นามบัตรเคลือบยา สามารถทำให้สลบได้ในทันทีหากสัมผัส', 'อย. ยกเลิกเลขสารบบอาหาร ผลิตภัณฑ์เสริมอาหาร เซเว่น เดย์ เซเว่น ดีบริษัทจากประเทศสวิตเซอร์แลนด

In [25]:
len(original_text_and_headline_news)

3716

--------------------------------------------------------------------------------------------------------------------------------------------------------------

In [26]:
original_and_headline_tokens_list = [split_word(txt) for txt in original_text_and_headline_news]

In [27]:
tokens_list_j = [','.join(tkn) for tkn in original_and_headline_tokens_list]

In [28]:
tvec = TfidfVectorizer(analyzer=lambda x:x.split(','),)
original_c_feat = tvec.fit_transform(tokens_list_j)

In [29]:
print(original_c_feat)

  (0, 1570)	0.36033805029143656
  (0, 4074)	0.33767509138823004
  (0, 975)	0.31159772876040714
  (0, 2347)	0.3783266527699001
  (0, 1264)	0.17943666395093094
  (0, 964)	0.34757493371733406
  (0, 1094)	0.2596871050975979
  (0, 1059)	0.18828382466336183
  (0, 1505)	0.34757493371733406
  (0, 1771)	0.21934257160270906
  (0, 5820)	0.22893538604503186
  (0, 364)	0.19667961269950246
  (1, 3300)	0.2596329224155616
  (1, 4477)	0.30984882536099273
  (1, 5811)	0.26402735176607844
  (1, 934)	0.31745318450260224
  (1, 3119)	0.19977769563511533
  (1, 4762)	0.2124438344072976
  (1, 1729)	0.2471160092146845
  (1, 5775)	0.13850454536898058
  (1, 1733)	0.33875895640451303
  (1, 3376)	0.23511722072255814
  (1, 3589)	0.3034194313170703
  (1, 5000)	0.26893990582534605
  (1, 1094)	0.24413556282335802
  :	:
  (3714, 1796)	0.2980774194723191
  (3714, 3529)	0.2980774194723191
  (3714, 3259)	0.2980774194723191
  (3714, 1520)	0.26321497302207186
  (3714, 4056)	0.2227055367021379
  (3714, 1491)	0.4333070889247863

In [30]:
original_text_and_headline_news_vector = original_c_feat[:,:].todense()
print(original_text_and_headline_news_vector)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [31]:
original_text_vector = original_text_and_headline_news_vector[::2]
headline_news_vector = original_text_and_headline_news_vector[1::2]

In [32]:
len(original_text_vector)

1858

In [33]:
len(headline_news_vector)

1858

In [34]:
find_cosine_similarity(original_text_vector, headline_news_vector, 0,0)

0.1769725012645834

In [35]:
original_text_vector

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [36]:
headline_news_vector

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [54]:
# similarity_matrix = []
# def create_similarity_matrix(original_text_vector, headline_news_vector):
#     for text in range(len(original_text_vector)):
#         tmp_array = []
#         for headline in range(len(headline_news_vector)):
#             tmp_array.append(find_cosine_similarity(original_text_vector, headline_news_vector, text, headline))
#         similarity_matrix.append(tmp_array)
    
#     df = pd.DataFrame(similarity_matrix, columns =[i for i in range(len(original_text_vector))], dtype = float)
#     return df


for j in range(100):
        similarity_matrix = []
        for text in range(1):
                similarity_matrix.append(cosine_similarity(original_text_vector[text], headline_news_vector[0:]).flatten()) # #cosine_similarity(original_text_vector[text],headline_news_vector[headline])
                # print(tmp_array)
                print(text)


df_similarity_matrix = pd.DataFrame([vector for vector in similarity_matrix], columns =[i for i in range(len(headline_news_vector))], dtype = float)
# df_similarity_matrix = pd.DataFrame(similarity_matrix, columns =[i for i in range(len(original_text_vector))], dtype = float)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [38]:
similarity_matrix

[array([0.1769725, 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ])]

In [39]:
# df_similarity_matrix = create_similarity_matrix(original_text_vector, headline_news_vector)

In [40]:
len(similarity_matrix)

1

In [41]:
cm = sns.light_palette("green", as_cmap=True)
df_similarity_matrix.style.background_gradient(cmap=cm)

# นำข่าวทั้งหมดมาจับคู่ข้ามกันเพื่อเตรียมข้อมูลสำหรับการ Train

## สร้างตารางขนาด row = จำนวนข่าว*จำนวนข่าว และ column = 3 (ข้อความดั้งเดิม / หัวข่าวอ้างอิง / เฉลย [คู่กัน = 1, คนละข่าว = 0])

In [42]:
original_text_and_headline_news_vector.shape[1]

5895

In [43]:
def create_dataset(original_text_vector, headline_news_vector):
    dataset = []
    for text in range(len(original_text_vector)):
        tmp = []
        for each_word in range(original_text_and_headline_news_vector.shape[1]):
            tmp.append(original_text_vector[text, each_word])
            tmp.append(headline_news_vector[text, each_word])
        tmp.append(1)
        dataset.append(tmp)

    headline = text
    for random_match in range(len(original_text_vector)):
        tmp = []
        while(text == headline):
            text = random.randint(1, len(original_text_vector))
            headline = random.randint(1, len(headline_news_vector))

        for each_word in range(original_text_and_headline_news_vector.shape[1]):
            tmp.append(original_text_vector[text, each_word])
            tmp.append(headline_news_vector[headline, each_word])
        tmp.append(0)
        dataset.append(tmp) 

    df = pd.DataFrame(dataset, columns =[i for i in range(original_text_and_headline_news_vector.shape[1]*2+1)], dtype = float)
    return df

In [44]:
df_dataset = create_dataset(original_text_vector, headline_news_vector)

In [45]:
df_dataset

,0,1,2,3,4,5,6,7,8,9,...,11781,11782,11783,11784,11785,11786,11787,11788,11789,11790
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3711,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
original_text_and_headline_news_vector.shape[1]

5895

In [47]:
X = df_dataset.drop(df_dataset.columns[(original_text_and_headline_news_vector.shape[1]*2)],axis=1)
y = df_dataset[df_dataset.columns[(original_text_and_headline_news_vector.shape[1]*2)]]

# แบ่งข้อมูลเป็น Train/Test

In [48]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.3, random_state = 100, stratify=y)

# ใช้ model SVM

In [49]:
clf_svm = svm.SVC(kernel='rbf')
clf_svm.fit(X_train, y_train)

SVC()

In [50]:
y_pred = clf_svm.predict(X_test)  
y_pred

array([0., 0., 1., ..., 1., 0., 0.])

In [51]:
print('Non Standardize')
print('Accuracy of SVM classifier on training set: {:.4f}'
     .format(clf_svm.score(X_train, y_train)*100))
print('Accuracy of SVM classifier on test set: {:.4f}'
     .format(clf_svm.score(X_test, y_test)*100))

Non Standardize
Accuracy of SVM classifier on training set: 100.0000
Accuracy of SVM classifier on test set: 100.0000


In [52]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       558
         1.0       1.00      1.00      1.00       557

    accuracy                           1.00      1115
   macro avg       1.00      1.00      1.00      1115
weighted avg       1.00      1.00      1.00      1115

[[558   0]
 [  0 557]]


# ใช้ model Random forest

In [53]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

# prediction on test set
y_pred=clf.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

Accuracy: 1.0
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       558
         1.0       1.00      1.00      1.00       557

    accuracy                           1.00      1115
   macro avg       1.00      1.00      1.00      1115
weighted avg       1.00      1.00      1.00      1115

[[558   0]
 [  0 557]]
